### Examples Of Pyspark ML

#### INICIAMOS SESION

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dataframe').getOrCreate()

In [2]:
spark

CARGAMOS CSV

In [3]:
training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [4]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



En PySpark se trabaja de forma diferente. Tendremos que agrupar nuestras variables independientes de forma que queden todas en una columna y dentro de una lista, por lo que crearemos un vector de ensamblaje o "vector assembler", de tal modo que queden así esas variables independientes:
- [Age, Experience]

Lo que haremos con estas dos, será tratarlas como una nueva variable independiente:
- [Age, Experience] ----> nueva_variable_independiente

In [5]:
from pyspark.ml.feature import VectorAssembler 

In [6]:
feature_assembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol='Independent features') # 1:46:27 del vídeo

In [7]:
output = feature_assembler.transform(training)

Veremos que se crea una nueva columna cuyos valores se corresponden a unos array con el contenido de aquellas variables independientes que hemos agrupado. Esto será nuestro input feature o lo que solíamos definir como train.

In [8]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



Seleccionamos las columnas que nos interesan para nuestro modelo: el train (Independent Features) y el test (Salary)

In [11]:
finalized_data = output.select('Independent features', 'Salary')
finalized_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



A continuación, entrenaremos un modelo de regresión lineal.

In [9]:
from pyspark.ml.regression import LinearRegression

In [12]:
train, test = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent features', labelCol='Salary')
regressor = regressor.fit(train)

In [13]:
regressor.coefficients

DenseVector([2666.6667, -2000.0])

In [14]:
regressor.intercept

-38833.33333337073

In [15]:
prediction = regressor.evaluate(test)

In [16]:
prediction.predictions.show()

c:\Users\Miguel Angel\AppData\Local\Programs\Python\Python37\lib\site-packages\pyspark\sql\context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [29.0,4.0]| 20000| 30500.00000000719|
|         [31.0,10.0]| 30000|23833.333333330404|
+--------------------+------+------------------+



In [17]:
# Errores

prediction.meanAbsoluteError, prediction.meanSquaredError

(8333.333333338393, 74138888.88898243)